<a href="https://colab.research.google.com/github/freddyduitama/GVD/blob/master/0_2_Introduction_To_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configuración plataforma

In [0]:
# instala el ambiente de spark..solo se corre una vez
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar xf spark-2.4.0-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
#Configura variables de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"

In [0]:
#import 
import findspark
findspark.init()
from pyspark.sql import SparkSession 
from pyspark import SparkContext,SparkConf

In [0]:
# definir la sesion. Note que Inicia con pyspark 
conf = SparkConf().setAppName("ejemplo").setMaster("local[*]")
sc = SparkContext(conf=conf)

In [0]:
#inicia sesion en el cluster. Note que inicia con pyspark.sql
spark = SparkSession.builder.master("local[*]").getOrCreate()

# ejemplo 1 b: MapPartition vs Map

In [0]:
# define función a usar en mapPartition
def f(iterator): yield sum(iterator)

In [0]:
one_through_9 = range(1,10)
print(one_through_9)

range(1, 10)


In [0]:
#  MapPartition with Three partitions. (1,2,3), (4,5,6), (7,8,9)
parallel = sc.parallelize(one_through_9,3)
parallel.mapPartitions(f).collect()

[6, 15, 24]

In [0]:
# MapPartition with two partitions (default).  (1,2,3,4), (5,6,7,8,9)
parallel = sc.parallelize(one_through_9)
parallel.mapPartitions(f).collect()

[10, 35]

In [0]:
#  Map with three partitions
parallel = sc.parallelize(one_through_9, 3)
parallel.map(lambda s: s)
parallel = parallel.reduce(lambda a, b : a + b)
print(parallel)

45


# **ejemplo 1 c : map reduce writting to disk**

In [0]:
# monta el google drive para usar sus archivos. Se corre cada que agregue archivos nuevos al Drive dirante la sesion
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
#  INST A: Opcional..si quiere subir archivos desde su PC  al ambiente de trabajo
from google.colab import files
datafile = files.upload()

In [0]:
#Step 1: Create a RDD with 4 partitions
lines = sc.textFile("/gdrive/My Drive/Colab Notebooks/Data/Pride_and_Prejudice.txt", 4)

In [0]:
#Step 2: Defining  transformations. Lengthg for each line in RDD
lineLengths = lines.map( lambda s: len(s) ) 

In [0]:
lineLengths.saveAsTextFile("cuenta")

In [0]:
lines1 = sc.textFile("cuenta", 4)

In [0]:
totalLength = lines1.map(lambda s: int(s)).reduce( lambda a, b : a + b)

In [0]:
print(totalLength)

671738


In [0]:
# lista archivos creados por saveAsTextFile
!ls  /content/cuenta

part-00000  part-00001	part-00002  part-00003	_SUCCESS


In [0]:
# muestra contenido de un archivo
!cat /content/cuenta/part-00001

cat: /content/cuenta/part-00001: No such file or directory


In [0]:
# borra directorio creado por saveAsTextFile
!rm -r cuenta

# Ejemplo   2   Filter

In [0]:
#  INST A: Opcional..si quiere subir archivos al ambiente de trabajo
from google.colab import files
datafile = files.upload()

Saving comments.txt to comments.txt


In [0]:
def isComment(s) : 
    if s.startswith('#'): return 1
    else : return 0     

In [0]:
# cuenta las lineas con comentario en el archivo
lines = sc.textFile("/gdrive/My Drive/Colab Notebooks/Data/comments.txt", 4)
comments = lines.filter(isComment)
comments.count()

1210

In [0]:
print(lines.count())

12482


# **Ejemplo 3** Key Value

In [0]:
#  INST A: Opcional..si quiere subir archivos al ambiente de trabajo
from google.colab import files
datafile = files.upload()

In [0]:
adult = sc.textFile("/gdrive/My Drive/Colab Notebooks/Data/clave-valor.txt")

In [0]:
adult1 = adult.map(lambda s: (s,1)).reduceByKey(lambda a,b : a + b)

In [0]:
adult1.take(10)

[('10', 2), ('70', 2), ('20', 1), ('89', 2), ('35', 1)]

# **Ejemplo 4:  Map to create key value pair

In [0]:
#  INST A: Opcional..si quiere subir archivos al ambiente de trabajo
from google.colab import files
datafile = files.upload()

In [0]:
def f(s): 
     k,v = s.split(" ")
     pair= (k,int(v))
     return pair

In [0]:
one_RDD = sc.textFile("/gdrive/My Drive/Colab Notebooks/Data/crear-clave.txt")

In [0]:
keyValue = one_RDD.map(f).reduceByKey(lambda a,b : a + b)

In [0]:
keyValue.take(6)

[('10', 8), ('70', 10), ('20', 3), ('89', 14), ('35', 4)]

In [0]:
!cat /gdrive/'My Drive'/'Colab Notebooks'/Data/crear-clave.txt

10 6
89 3
70 2
35 1
10 2
89 7
70 8
20 3
89 4
35 3


# Crear tipo Row  desde una matriz

In [0]:
import configparser
from pyspark import StorageLevel
from pyspark.sql import Row

In [0]:
#Función para usar en el map. 
# Toma cada file de la matriz, hace split sobre cada posición y la agrega a una lista con tipo float# Entra text file ['1 0 2', '3 1 2', '1 2 1']   
# Sale RDD [([1.0, 0.0, 2.0]), ([3.0, 1.0, 2.0]), ([1.0, 2.0, 1.0])]

def ProcesaFila(line):    
    ListValues=[]
    records = line.split()
    for item in records :
         ListValues.append(float(item))
    return ListValues

In [0]:
# Una linea es una fila de la matriz. 
# Ejemplo: Entrada La fila 2 con tres columnas tiene una estructura como ([1.0, 2.0, 1.0], 2)
# Salida Row(columna=2, fila=2, valor=2.0)
def CreateRow(line):    
    items=line[0]
    row=line[1]
    listTuplas=[]
    col=0
    for item in items :  
        listTuplas.append( Row(columna=col,fila=row,valor=item) )
        col=col+1
    return listTuplas

In [0]:
!ls /gdrive/'My Drive'/'Colab Notebooks'/Data

2015-summary.csv   crear-clave.txt  matriz-B.txt	      valores.txt
2015-summary.json  datos.json	    Pride_and_Prejudice.txt
clave-valor.txt    datos.txt	    stopList.txt
comments.txt	   matriz-A.txt     texto-con-caracteres.txt


In [0]:
matrizAtxt = sc.textFile("/gdrive/My Drive/Colab Notebooks/Data/matriz-A.txt")

In [0]:
matrizAtxt.take(3)

['1 0 2', '3 1 2', '1 2 1']

In [0]:
# se crea una lista por cada fila de la matriz y se agrega el numero de la fila con zipWithIndex
matrizA=matrizAtxt.map(lambda x : ProcesaFila(x)).zipWithIndex()

In [0]:
matrizA.take(10)

[([1.0, 0.0, 2.0], 0), ([3.0, 1.0, 2.0], 1), ([1.0, 2.0, 1.0], 2)]

In [0]:
RowsA = matrizA.flatMap(lambda x: CreateRow(x))

In [0]:
RowsA.take(3)

[Row(columna=0, fila=0, valor=1.0),
 Row(columna=1, fila=0, valor=0.0),
 Row(columna=2, fila=0, valor=2.0)]